In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
ps4=pd.read_excel("tweets_ps4.xlsx", sheet_name="Treinamento")
ps4Teste=pd.read_excel("tweets_ps4.xlsx", sheet_name="Teste")

def limpador(frase):
    tweet=[]
    frase=frase.lower()
    frase=frase.split()
    for j in frase:
        if len(j)>0:
            lista=[",",";","(",")","[","]","\n","...","***",".","-","|"]
            for a in lista:
                j=j.replace(a," ")
            if j[0]!="@" and j[:4]!="http":
                tweet.append(j)
    return tweet

In [3]:
###Naive Bayes Treinamento

frases=[]
#Pega os Tweets da coluna treinamento limpa eles e adiciona a uma lista chamada Frases.
for i in ps4["Treinamento"]:
    frases.append(' '.join(limpador(i)))

#Substitui os Tweets "sujos" pelos "limpos", contidos na lista Frases, no DataFrame.
for i in range(len(ps4["Treinamento"])):
    ps4["Treinamento"][i]=frases[i]

In [4]:
#Calcula o numero de Tweets relevantes e irrelevantes 
I,R=ps4["Classificação"].value_counts()
T=I+R
print("Tweets Irrelevantes: {}".format(I))
print("Tweets Relevantes: {}".format(R))
print("Total={}".format(T))
# ps4Treina.sort_values(by="Classificação").head()
# ps4Treina.sort_values(by="Classificação").tail()

Tweets Irrelevantes: 184
Tweets Relevantes: 116
Total=300


In [5]:
palavrasR=[]
palavrasI=[]
# Separa cada palavra, dependendo da relevancia do Tweets em que está contida, e adiciona a uma lista conforme a relevancia
# Também cria uma lista com todas as palavras.
for i in range(len(ps4)):
    if ps4["Classificação"][i] == "relevante":
        fraseSplit=ps4["Treinamento"][i].split()
        palavrasR.extend(fraseSplit)
    else:
        fraseSplit=ps4["Treinamento"][i].split()
        palavrasI.extend(fraseSplit)
    
    
    
#Cria um novo dataframe com todas as palavras Relevantes e suas probabilidades.
palavrasRDF=pd.DataFrame(data=palavrasR)
palavrasRDF=palavrasRDF[0].value_counts().to_frame()
palavrasRDF.columns=["Quantidade de Relevantes"]


#Cria um novo dataframe com todas as palavras Irrelevantes e suas probabilidades.
palavrasIDF=pd.DataFrame(data=palavrasI)
palavrasIDF=palavrasIDF[0].value_counts().to_frame()
palavrasIDF.columns=["Quantidade de Irrelevantes"]


palavrasDF=palavrasIDF.join(palavrasRDF, how="outer").fillna(0)
palavrasDF.sort_values(by="Quantidade de Irrelevantes", ascending=False).head()

#palavrasRDF.head(10)
#palavrasIDF.head(10)
#palavrasDF.head(10)

,Quantidade de Irrelevantes,Quantidade de Relevantes
rt,123.0,34.0
the,115.0,73.0
ps4,110.0,115.0
a,87.0,49.0
to,78.0,43.0


In [6]:
def classificador(tweet):
    total=len(palavrasDF)
    somaRelevantes=sum(palavrasDF["Quantidade de Relevantes"])
    somaIrrelevantes=sum(palavrasDF["Quantidade de Irrelevantes"])
    
    probabilidadeRelevante=1
    probabilidadeIrrelevante=1
    for i in limpador(tweet):
        if i in palavrasDF.index:
            probabilidadeRelevante*=((palavrasDF["Quantidade de Relevantes"][i]+1)/(somaRelevantes+total))
            probabilidadeIrrelevante*=((palavrasDF["Quantidade de Irrelevantes"][i]+1)/(somaIrrelevantes+total))
        else:
            probabilidadeRelevante*=(1/(somaRelevantes+total))
            probabilidadeIrrelevante*=(1/(somaIrrelevantes+total))
            
    probabilidadeRelevante*=(R/T)
    probabilidadeIrrelevante*=(I/T)
    
    if probabilidadeRelevante>probabilidadeIrrelevante:
        return "relevante"
    else:
        return "irrelevante"
    

In [7]:
positivosFalso=0
positivosVerdadeiro=0
negativosVerdadeiro=0
negativosFalso=0

for k in ps4Teste.index:
    ps4Teste["Classificador"]=classificador(ps4Teste["Teste"][k])
    if ps4Teste["Classificação"][k]=="relevante":
        if ps4Teste["Classificação"][k]==classificador(ps4Teste["Teste"][k]):
            positivosVerdadeiro+=1  
        else:
            positivosFalso+=1
        
    elif ps4Teste["Classificação"][k]=="irrelevante":
        if ps4Teste["Classificação"][k]==classificador(ps4Teste["Teste"][k]):
            negativosVerdadeiro+=1
        else:
            negativosFalso+=1
            
        
        
        
print("Positivo Falso: {}, {}%".format(positivosFalso,(positivosFalso/2)))
print("Positivo Verdadeiro: {}, {}%".format(positivosVerdadeiro,(positivosVerdadeiro/2)))
print("Negativo Verdadeiro: {}, {}%".format(negativosVerdadeiro,(negativosVerdadeiro/2)))
print("Negativo Falso: {}, {}%".format(negativosFalso,(negativosFalso/2)))
print("Porcentagem de erro do Classificador: {}%".format((positivosFalso/2)+(negativosFalso/2)))


Positivo Falso: 4, 2.0%
Positivo Verdadeiro: 78, 39.0%
Negativo Verdadeiro: 98, 49.0%
Negativo Falso: 20, 10.0%
Porcentagem de erro do Classificador: 12.0%
